# OpenDART API를 활용한 기업 정보 수집 테스트

이 노트북은 OpenDART API의 주요 기능을 테스트하여 AI Agent 개발 시 기업 정보 수집 도구(Tool)에 어떤 정보를 요청하고 받을 수 있는지, 그리고 어떻게 연동할 수 있는지를 파악하는 데 도움을 주기 위해 작성되었습니다.

**OpenDART API란?**
OpenDART는 금융감독원(FSS)에서 제공하는 전자공시시스템(DART)의 공시정보를 API 형태로 제공하여 개발자들이 다양한 서비스와 애플리케이션에서 활용할 수 있도록 지원하는 서비스입니다. 기업의 개황 정보, 재무 정보, 지분 현황, 주요 공시 내용 등을 프로그래밍 방식으로 접근할 수 있습니다.

**API 키 발급 안내:**
OpenDART API를 사용하기 위해서는 먼저 [DART 개발자 사이트](https://opendart.fss.or.kr/)에서 API 인증키를 발급받아야 합니다. 발급받은 인증키를 아래 코드의 `YOUR_OPENDART_API_KEY` 부분에 입력하거나, 환경 변수 등을 통해 안전하게 관리하는 것이 좋습니다.

## 1. 기본 설정 및 라이브러리 임포트

API 요청에 필요한 라이브러리를 임포트하고, API 키 및 기본 URL을 설정합니다.

In [ ]:
import requests
import json
import pandas as pd # 데이터 표시에 용이
from IPython.display import display, Markdown # 마크다운 표시용

# 🚨 중요: 발급받은 OpenDART API 인증키를 입력하세요.
# 실제 운영 환경에서는 환경 변수나 별도의 설정 파일을 사용하는 것이 안전합니다.
API_KEY = "YOUR_OPENDART_API_KEY" 
BASE_URL = "https://opendart.fss.or.kr/api"

def handle_api_response(response):
    """API 응답을 처리하고 JSON 데이터를 반환하거나 오류를 출력합니다."""
    if response.status_code == 200:
        try:
            data = response.json()
            if data.get("status") == "000": # 정상 응답
                return data
            elif data.get("status") == "013": # 데이터 없음
                print(f"데이터 없음: {data.get('message')}")
                return None
            else:
                print(f"API 오류: {data.get('status')} - {data.get('message')}")
                return None
        except json.JSONDecodeError:
            print("JSON 디코딩 오류가 발생했습니다. 응답 내용:")
            print(response.text)
            return None
    else:
        print(f"HTTP 오류: {response.status_code}")
        print(response.text)
        return None

## 2. 기업 고유번호 조회 (corpCode.xml / corpCode.json)

회사명을 이용하여 DART에서 사용하는 고유번호(corp_code)를 조회합니다. 이 고유번호는 다른 API 호출 시 필수적으로 사용됩니다.
- **API Endpoint**: `/corpCode.json`
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_name`: 회사명 (예: "삼성전자")

In [ ]:
def get_corp_code(corp_name):
    """회사명을 기준으로 기업 고유번호를 조회합니다."""
    url = f"{BASE_URL}/corpCode.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_name': corp_name
    }
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data and data.get('list'):
        # 여러 회사가 검색될 수 있으므로, 첫 번째 회사를 선택하거나 사용자가 선택하도록 할 수 있습니다.
        # 여기서는 간단히 첫 번째 결과를 사용합니다.
        corp_info = data['list'][0]
        print(f"회사명: {corp_info['corp_name']}, 고유번호: {corp_info['corp_code']}, 종목코드: {corp_info.get('stock_code', 'N/A')}")
        return corp_info['corp_code']
    else:
        print(f"'{corp_name}'에 대한 기업 고유번호를 찾을 수 없습니다.")
        return None

# 테스트할 회사명
company_name_to_test = "삼성전자"
samsung_corp_code = get_corp_code(company_name_to_test)

company_name_to_test_2 = "현대자동차"
hyundai_corp_code = get_corp_code(company_name_to_test_2)

## 3. 기업 개황 정보 조회 (company.json)

기업의 상세한 일반 현황(영문 명칭, 주소, 대표자명, 주요 사업 등)을 조회합니다.
- **API Endpoint**: `/company.json`
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호 (앞 단계에서 획득)

In [ ]:
def get_company_overview(corp_code):
    """기업 고유번호를 사용하여 기업 개황 정보를 조회합니다."""
    if not corp_code:
        print("기업 고유번호가 없어 조회할 수 없습니다.")
        return None
        
    url = f"{BASE_URL}/company.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code
    }
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data:
        print(f"\n--- [{data.get('corp_name', 'N/A')}] 기업 개황 정보 ---")
        overview_df = pd.DataFrame([data]) # 리스트로 감싸서 DataFrame 생성
        # 필요없는 컬럼이나 순서 조정 가능
        overview_df = overview_df[['corp_name', 'corp_cls', 'jurir_no', 'bizr_no', 'adres', 'hm_url', 'phn_no', 'est_dt', 'acc_mt']]
        display(overview_df)
        return data
    return None

# 삼성전자 기업 개황 정보 조회 테스트
if samsung_corp_code:
    samsung_overview = get_company_overview(samsung_corp_code)

## 4. 주요 재무정보 조회 (fnlttSinglAcnt.json / fnlttMultiAcnt.json)

기업의 특정 사업연도 및 보고서 코드에 따른 주요 재무정보(자산, 부채, 자본, 매출액, 영업이익, 당기순이익 등)를 조회합니다.

**보고서 코드 (reprt_code)**:
- `11013`: 1분기보고서
- `11012`: 반기보고서
- `11014`: 3분기보고서
- `11011`: 사업보고서 (연간)

**API Endpoint**: `/fnlttSinglAcnt.json` (주요 단일계정) 또는 `/fnlttMultiAcnt.json` (다중계정, 더 상세)
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호
  - `bsns_year`: 사업연도 (예: "2023")
  - `reprt_code`: 보고서 코드

In [ ]:
def get_financial_statement(corp_code, bsns_year, reprt_code="11011"):
    """기업의 특정 연도, 보고서 종류에 따른 재무제표를 조회합니다."""
    if not corp_code:
        print("기업 고유번호가 없어 조회할 수 없습니다.")
        return None
        
    # 여기서는 fnlttSinglAcnt.json (주요계정)을 사용합니다.
    # 더 상세한 정보를 원하면 fnlttMultiAcnt.json을 사용할 수 있습니다.
    url = f"{BASE_URL}/fnlttSinglAcnt.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': bsns_year,
        'reprt_code': reprt_code
        # 'fs_div': 'CFS' # 연결재무제표. 'OFS'는 개별재무제표
    }
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data and data.get('list'):
        print(f"\n--- [{corp_code}] {bsns_year}년 {reprt_code} 재무 정보 ---")
        fs_df = pd.DataFrame(data['list'])
        # 필요에 따라 컬럼 선택 및 이름 변경
        # 예: 'account_nm', 'thstrm_nm', 'thstrm_amount', 'frmtrm_amount'
        display(fs_df[['sj_nm', 'account_nm', 'account_detail', 'thstrm_nm', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount']])
        return data['list']
    else:
        print(f"[{corp_code}] {bsns_year}년 {reprt_code} 재무 정보를 찾을 수 없습니다.")
    return None

# 삼성전자 2023년 사업보고서(연간) 재무 정보 조회 테스트
if samsung_corp_code:
    samsung_fs_2023_annual = get_financial_statement(samsung_corp_code, "2023", "11011")

# 현대자동차 2023년 3분기 보고서 재무 정보 조회 테스트
if hyundai_corp_code:
    hyundai_fs_2023_q3 = get_financial_statement(hyundai_corp_code, "2023", "11014")

## 5. 대주주 현황 조회 (majorstockholders.json)

기업의 대주주(최대주주 및 주요주주)에 대한 정보를 조회합니다.
- **API Endpoint**: `/majorstockholders.json`
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호

In [ ]:
def get_major_shareholders(corp_code):
    """기업의 대주주 현황을 조회합니다."""
    if not corp_code:
        print("기업 고유번호가 없어 조회할 수 없습니다.")
        return None
        
    url = f"{BASE_URL}/majorstockholders.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code
    }
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data and data.get('list'):
        print(f"\n--- [{corp_code}] 대주주 현황 ---")
        shareholders_df = pd.DataFrame(data['list'])
        display(shareholders_df[['rcept_dt', 'repror', 'stkqy', 'stkqy_irds', 'stkrt', 'stkrt_irds']])
        return data['list']
    else:
        print(f"[{corp_code}] 대주주 현황 정보를 찾을 수 없습니다.")
    return None

# 삼성전자 대주주 현황 조회 테스트
if samsung_corp_code:
    samsung_shareholders = get_major_shareholders(samsung_corp_code)

## 6. 최근 공시 목록 조회 (list.json)

특정 기업의 최근 공시된 문서 목록을 조회합니다. 기간, 공시 유형 등 다양한 조건으로 필터링할 수 있습니다.
- **API Endpoint**: `/list.json`
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호
- **선택 파라미터 (예시)**:
  - `bgn_de`: 검색 시작일 (YYYYMMDD)
  - `end_de`: 검색 종료일 (YYYYMMDD)
  - `pblntf_ty`: 공시유형 (A: 정기공시, B: 주요사항보고, C: 발행공시 등. 상세 코드는 OpenDART 명세 참조)
  - `page_no`: 페이지 번호 (기본값: 1)
  - `page_count`: 페이지당 건수 (기본값: 10, 최대: 100)

In [ ]:
def get_recent_disclosures(corp_code, bgn_de=None, end_de=None, pblntf_ty=None, page_no=1, page_count=10):
    """기업의 최근 공시 목록을 조회합니다."""
    if not corp_code:
        print("기업 고유번호가 없어 조회할 수 없습니다.")
        return None
        
    url = f"{BASE_URL}/list.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'page_no': page_no,
        'page_count': page_count
    }
    if bgn_de: params['bgn_de'] = bgn_de
    if end_de: params['end_de'] = end_de
    if pblntf_ty: params['pblntf_ty'] = pblntf_ty
    
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data and data.get('list'):
        print(f"\n--- [{corp_code}] 최근 공시 목록 (Page: {data.get('page_no', 'N/A')}/{data.get('total_page', 'N/A')}) ---")
        disclosures_df = pd.DataFrame(data['list'])
        # 각 공시 문서는 'rcept_no' (접수번호)를 통해 원문 조회가 가능합니다.
        display(disclosures_df[['rcept_dt', 'corp_name', 'report_nm', 'rcept_no', 'flr_nm']])
        # 공시 원문 URL 예시 (실제로는 document.xml API를 통해 접근)
        # for index, row in disclosures_df.head(1).iterrows(): # 첫 번째 공시만 예시
        #    doc_url = f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={row['rcept_no']}"
        #    display(Markdown(f"[공시 원문 보기]({doc_url})"))
        return data['list']
    else:
        print(f"[{corp_code}] 공시 목록 정보를 찾을 수 없습니다.")
    return None

# 삼성전자 최근 5건 공시 조회 테스트 (오늘 날짜 기준, 필요시 날짜 지정)
from datetime import datetime, timedelta
today = datetime.today().strftime('%Y%m%d')
one_month_ago = (datetime.today() - timedelta(days=30)).strftime('%Y%m%d')

if samsung_corp_code:
    # 최근 30일간의 공시 중 상위 5개
    samsung_disclosures = get_recent_disclosures(samsung_corp_code, 
                                                 bgn_de=one_month_ago, 
                                                 end_de=today, 
                                                 page_count=5)

# 현대자동차 정기공시(A) 최근 2건 조회 테스트
if hyundai_corp_code:
    hyundai_periodic_disclosures = get_recent_disclosures(hyundai_corp_code, 
                                                          pblntf_ty='A', 
                                                          page_count=2)

## 7. 공시서류 원문 조회 (document.xml)

공시 목록에서 얻은 `rcept_no` (접수번호)를 사용하여 공시 서류의 원문(주로 HTML 또는 XML 형식의 뷰어 페이지)에 접근할 수 있는 URL 정보를 얻습니다. 
실제 원문 파일 다운로드가 아니라, DART 뷰어 페이지로 연결되는 URL을 제공하는 경우가 많습니다. 반환 형식은 XML입니다.

- **API Endpoint**: `/document.xml` (XML 응답)
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `rcept_no`: 접수번호 (공시목록 조회 API에서 획득)

**참고:** 이 API는 XML을 반환하므로, 파싱을 위해 `xml.etree.ElementTree` 같은 라이브러리가 필요할 수 있습니다. 여기서는 간단히 DART 뷰어 URL을 구성하는 방법을 안내합니다.

In [ ]:
def get_disclosure_document_viewer_url(rcept_no):
    """접수번호를 사용하여 공시문서 뷰어 URL을 반환합니다 (실제 API 호출은 생략하고 URL 구조만 안내)."""
    if not rcept_no:
        print("접수번호가 없어 URL을 생성할 수 없습니다.")
        return None
    
    # OpenDART의 document.xml API는 해당 접수번호의 문서 내용을 직접 반환하거나, 
    # 문서 내의 파일 목록(zip 파일 내의 dcm 파일 등)을 알려줍니다.
    # 가장 간단하게는 DART 웹사이트의 뷰어 URL을 구성할 수 있습니다.
    viewer_url = f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}"
    print(f"접수번호 [{rcept_no}]의 공시문서 DART 뷰어 URL: {viewer_url}")
    display(Markdown(f"[접수번호 {rcept_no} 공시문서 보기]({viewer_url})"))
    return viewer_url

# 이전 단계에서 얻은 공시 목록 중 하나의 접수번호를 사용하여 테스트
if samsung_disclosures and len(samsung_disclosures) > 0:
    sample_rcept_no = samsung_disclosures[0]['rcept_no']
    get_disclosure_document_viewer_url(sample_rcept_no)
else:
    print("\n테스트할 공시 정보가 없습니다. 이전 단계의 get_recent_disclosures를 먼저 실행해주세요.")

## 8. 결론 및 활용 방안

이 노트북을 통해 OpenDART API의 주요 기능을 테스트하고, AI Agent 개발에 필요한 기업 정보를 어떻게 수집할 수 있는지 확인했습니다.

**수집 가능한 주요 정보:**
- 기업 고유번호
- 기업 일반 현황 (회사명, 주소, 대표자, 사업목적 등)
- 재무제표 (자산, 부채, 자본, 손익계산서 주요 항목 등)
- 대주주 및 지분 현황
- 최근 공시 목록 및 공시 원문 접근 정보

**AI Agent Tool 개발 시 고려사항:**
- **Tool의 입력**: 사용자로부터 회사명, 조회 기간, 특정 정보 유형(재무, 공시 등)을 입력받을 수 있습니다.
- **Tool의 처리**: 입력값을 바탕으로 이 노트북에서 테스트한 API들을 순차적 또는 조건부로 호출합니다.
  - 예: 회사명 -> `get_corp_code` -> `corp_code` 획득 -> `get_financial_statement` 또는 `get_recent_disclosures` 호출.
- **Tool의 출력**: 조회된 정보를 가공하여 사용자에게 친화적인 형태로 제공합니다. (텍스트 요약, 표, 차트 등)
- **오류 처리 및 API 제한**: API 응답 상태(status)를 확인하고, 데이터가 없거나(013) 오류 발생 시 적절히 처리해야 합니다. 또한, OpenDART API는 일일 사용량 제한이 있으므로 효율적인 호출 관리가 필요합니다.

이 테스트 코드를 기반으로 실제 AI Agent의 Tool을 구현하고, 다양한 기업 정보를 분석하여 투자 결정 지원, 시장 동향 파악 등 복잡한 작업을 수행하는 Agent를 구축할 수 있을 것입니다.